<a href="https://colab.research.google.com/github/mk-268064/Machine-Learning/blob/main/run2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install deepproblog

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 545.6/545.6 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [8]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from deepproblog.model import Model
import torch

# Load dataset
data = {
    'product_type': ['electronics'] * 10 + ['clothing'] * 10 + ['furniture'] * 10,
    'origin_country': ['China', 'USA', 'China', 'Germany', 'China', 'USA', 'India', 'Vietnam', 'Japan', 'Korea'] * 3,
    'destination_country': ['USA', 'China', 'Germany', 'USA', 'Germany', 'USA', 'USA', 'China', 'Japan', 'Korea'] * 3,
    'value': [500, 700, 200, 300, 800, 450, 150, 600, 350, 900] * 3,
    'risk_level': [1, 2, 1, 0, 2, 1, 0, 2, 1, 2] * 3
}
df = pd.DataFrame(data)

# Assign country risk scores
country_risk = {'China': 3, 'Vietnam': 2, 'India': 2, 'USA': 1, 'Germany': 1, 'Japan': 1, 'Korea': 1}
df['origin_risk_score'] = df['origin_country'].map(country_risk)
df['destination_risk_score'] = df['destination_country'].map(country_risk)

# Assign product risk scores
product_risk = {'electronics': 3, 'clothing': 1, 'furniture': 1}
df['product_risk_score'] = df['product_type'].map(product_risk)

# High-value shipment flag
value_threshold = df['value'].quantile(0.75)
df['is_high_value'] = (df['value'] > value_threshold).astype(int)

# High-risk & high-value shipments
df['high_risk_high_value'] = ((df['origin_risk_score'] >= 2) & (df['is_high_value'] == 1)).astype(int)

# Shipment count & large shipment flag
df['shipment_count'] = df.groupby('origin_country')['origin_country'].transform('count')
df['is_large_shipment'] = (df['shipment_count'] > df['shipment_count'].median()).astype(int)

# Feature creation function
def create_features(df):
    df['value_category'] = pd.qcut(df['value'], q=5, labels=['VL', 'L', 'M', 'H', 'VH'])
    df['high_risk_route'] = ((df['origin_country'] == 'China') &
                              (df['destination_country'].isin(['USA', 'Germany']))).astype(int)
    return df

df = create_features(df)

# Encode categorical variables
le_dict = {}
categorical_columns = ['product_type', 'origin_country', 'destination_country', 'value_category']
for col in categorical_columns:
    le_dict[col] = LabelEncoder()
    df[f'{col}_encoded'] = le_dict[col].fit_transform(df[col])

# Prepare dataset for training
X = df.drop(columns=['risk_level', 'product_type', 'origin_country', 'destination_country', 'value_category'])
y = df['risk_level']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train XGBoost
xgb_model = XGBClassifier(n_estimators=300, learning_rate=0.05, max_depth=10, random_state=42)
xgb_model.fit(X_train_scaled, y_train)

# XGBoost predictions
xgb_preds = xgb_model.predict(X_test_scaled)
xgb_confidence = xgb_model.predict_proba(X_test_scaled).max(axis=1)

# Load Neuro-Symbolic Model
neuro_symbolic_model = Model("path/to/your/problog_program.pl", load=True)
neuro_symbolic_model.load_state_dict(torch.load("deep_problog_risk_model_state_dict.pt"))
# Convert test data to Prolog format and get predictions
symbolic_preds = []
for row in X_test.itertuples():
    symbolic_pred = neuro_symbolic_model.predict(f"shipment_risk({row.Index})")
    symbolic_preds.append(int(symbolic_pred))

# Combine predictions based on confidence
def combined_risk(xgb_risk, symbolic_risk, confidence):
    if confidence > 0.8:
        return xgb_risk
    elif confidence > 0.5:
        return int(0.7 * xgb_risk + 0.3 * symbolic_risk)
    else:
        return symbolic_risk

final_risks = np.array([
    combined_risk(xgb_preds[i], symbolic_preds[i], xgb_confidence[i])
    for i in range(len(X_test))
])

# Store results
X_test['final_risk'] = final_risks

# Save final results
X_test.to_csv("final_risk_predictions.csv", index=False)

print("Hybrid risk predictions saved to final_risk_predictions.csv")


NameError: name 'neuro_symbolic_model' is not defined